In [ ]:
import itertools
import json
import os

def run_benchmarks(**kwargs):
    def with_default_kwargs(bench_environment={}, worker_environment={}, **kwargs):
        return {
            **dict(
                rate=2., iterations=100, parallelism=8, stream_length=50400, local_acker_flush_delay_in_millis=5,
                bench_environment={**dict(WARM_UP_STREAM_LENGTH="10080", WARM_UP_DELAY_MS="10"), **bench_environment},
                worker_environment={**dict(BARRIER_DISABLED="TRUE", **worker_environment)},
            ),
            **kwargs,
        }

    def print_kwargs(kwargs):
        print(kwargs)
        return kwargs    

    def with_results_name(results_name=None, **kwargs):
        def value(iterations=None, rate=None, **_):
            return f"sigmod.07.30.idle-cpu-level=5/iterations={iterations}/rate={rate}/{results_name}"    

        return dict(results_name=value(**kwargs), **kwargs)

    !ansible-playbook --extra-vars '{json.dumps(with_results_name(**print_kwargs(with_default_kwargs(**kwargs))))}' -i aws.yml flamestream.yml

In [ ]:
for LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS, LOCAL_ACKER_FLUSH_COUNT in itertools.chain([(2, 2000)], itertools.product([5, 10], [500, 1000, 2000])):
    run_benchmarks(
        results_name=f"distributed=True/LOCAL_ACKER_FLUSH_COUNT={LOCAL_ACKER_FLUSH_COUNT}/LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS={LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS}",
        tracking_frequency=1,
        tracking="acking",
        distributed_acker=True,
        rate=2,
        bench_environment=dict(WARM_UP_STREAM_LENGTH=50000),
        worker_environment=dict(
            LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS=LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS,
            LOCAL_ACKER_FLUSH_COUNT=LOCAL_ACKER_FLUSH_COUNT,
        ),
    )

In [27]:
run_benchmarks(
    parallelism=2,
    results_name="tracking_frequency=0/tracking=disabled", tracking_frequency=0, tracking="disabled", rate=10, iterations=1000,
    bench_environment=dict(WARM_UP_STREAM_LENGTH=0, WARM_UP_DELAY_MS=20)
)
# for distributed in [False, True]:
#     run_benchmarks(
#         results_name=f"LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS=2/tracking_frequency=1/tracking=acking/distributed={distributed}",
#         tracking_frequency=1, iterations=1000,
#         tracking="acking",
#         distributed_acker=distributed,
#         rate=10,
#         bench_environment=dict(WARM_UP_STREAM_LENGTH=50000, WARM_UP_DELAY_MS=20),
#         worker_environment=dict(LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS=5, LOCAL_ACKER_FLUSH_COUNT=2000),
#     )
# run_benchmarks(
#     results_name=f"tracking_frequency={tracking_frequency}/tracking=watermarking",
#     tracking_frequency=1,
#     tracking="watermarking",
#     rate=2
# )

{'rate': 10, 'iterations': 1000, 'parallelism': 2, 'stream_length': 50400, 'local_acker_flush_delay_in_millis': 5, 'bench_environment': {'WARM_UP_STREAM_LENGTH': 0, 'WARM_UP_DELAY_MS': 20}, 'worker_environment': {'BARRIER_DISABLED': 'TRUE'}, 'results_name': 'tracking_frequency=0/tracking=disabled', 'tracking_frequency': 0, 'tracking': 'disabled'}

PLAY [Install Python] **********************************************************

TASK [Install Python 2.x] ******************************************************
changed: [10.0.0.71]
changed: [10.0.0.216]
changed: [10.0.0.200]
changed: [10.0.0.157]
changed: [10.0.0.156]
changed: [10.0.0.51]
changed: [10.0.0.193]
changed: [10.0.0.114]
changed: [10.0.0.117]
changed: [10.0.0.203]
changed: [10.0.0.110]

PLAY [Prepare cluster] *********************************************************

TASK [Gathering Facts] *********************************************************
ok: [10.0.0.157]
ok: [10.0.0.156]
ok: [10.0.0.216]
ok: [10.0.0.200]
ok: [10.0.0.71]

In [13]:
for LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS in [2, 5, 10]:
#     run_benchmarks(results_name="tracking_frequency=0/tracking=disabled", tracking_frequency=0, tracking="disabled", rate=rate,
#                    bench_environment=dict(WARM_UP_STREAM_LENGTH=50000))
    for LOCAL_ACKER_FLUSH_COUNT in [500, 1000, 2000]:
        for distributed in [False]:
            run_benchmarks(
                results_name=f"LOCAL_ACKER_FLUSH_COUNT={LOCAL_ACKER_FLUSH_COUNT}/LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS={LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS}",
                tracking_frequency=1,
                tracking="acking",
                distributed_acker=distributed,
                rate=2,
                bench_environment=dict(WARM_UP_STREAM_LENGTH=50000),
                worker_environment=dict(
                    LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS=LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS,
                    LOCAL_ACKER_FLUSH_COUNT=LOCAL_ACKER_FLUSH_COUNT,
                ),
            )
#         run_benchmarks(
#             results_name=f"tracking_frequency={tracking_frequency}/tracking=watermarking",
#             tracking_frequency=tracking_frequency,
#             tracking="watermarking",
#             rate=rate
#         )

{'rate': 2, 'iterations': 100, 'parallelism': 8, 'stream_length': 50400, 'local_acker_flush_delay_in_millis': 5, 'bench_environment': {'WARM_UP_STREAM_LENGTH': 50000, 'WARM_UP_DELAY_MS': '10'}, 'worker_environment': {'BARRIER_DISABLED': 'TRUE', 'LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS': 2, 'LOCAL_ACKER_FLUSH_COUNT': 500}, 'results_name': 'LOCAL_ACKER_FLUSH_COUNT=500/LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS=2', 'tracking_frequency': 1, 'tracking': 'acking', 'distributed_acker': False}

PLAY [Install Python] **********************************************************

TASK [Install Python 2.x] ******************************************************
changed: [10.0.0.71]
^C
 [ERROR]: User interrupted execution

{'rate': 2, 'iterations': 100, 'parallelism': 8, 'stream_length': 50400, 'local_acker_flush_delay_in_millis': 5, 'bench_environment': {'WARM_UP_STREAM_LENGTH': 50000, 'WARM_UP_DELAY_MS': '10'}, 'worker_environment': {'BARRIER_DISABLED': 'TRUE', 'LOCAL_ACKER_FLUSH_DELAY_IN_MILLIS': 2, 'LOCAL_ACKER_